## Get dataset to train

In [1]:
import json
import torch    #if error, restart kernel
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

import glob
import pandas as pd

Fs = 100  # Sampling frequency

# Path to the folder containing the CSV files
folder_path = "TFO Data\IMU"

# Get the list of CSV files in the folder
file_list = glob.glob(folder_path + "\*.csv")
file_path = file_list[0]

# Read the CSV file into a pandas dataframe
df = pd.read_csv(file_path)
df = df[df.columns[2:-1]]  # Remove the time, index, and temperature columns
    
# Show the head of the dataframe
title = file_path.split("\\")[-1]
print(title)
df.head()

P19V1_20.csv


,ax1,ay1,az1,gx1,gy1,gz1,ax2,ay2,az2,gx2,gy2,gz2
0,0.042,-0.434,-0.837,-1.717,1.022,0.816,0.133,0.005,-1.031,-3.113,1.747,-1.862
1,0.043,-0.437,-0.842,-1.846,0.916,0.496,0.135,0.005,-1.027,-2.853,1.717,-1.869
2,0.044,-0.434,-0.839,-1.717,1.289,0.687,0.133,0.004,-1.028,-2.350,1.785,-1.801
3,0.040,-0.438,-0.836,-1.724,1.106,0.732,0.135,0.005,-1.026,-2.396,1.831,-1.640
4,0.044,-0.438,-0.838,-1.671,1.022,0.534,0.135,0.003,-1.023,-2.335,1.923,-1.404


In [2]:
# from sklearn.preprocessing import StandardScaler

# # Normalize the data
# scaler = StandardScaler()
# df = scaler.fit_transform(df)

In [3]:
x = df['ax1']
print(x.shape)

(110172,)


## Create model and dataloaders

In [4]:
from Utils.models import VAE
from Utils.dataloader import DataLoaderGenerator
from Utils.processing import window_data

# Window the data
data = window_data(x, 100)
print(f"Data shape: {data.shape}")

# Create a DataLoader object
dataloader = DataLoaderGenerator(data, batch_size=32)
train_loader, val_loader = dataloader.generate()

# Initialize the model
vae = VAE(input_dims=100, latent_dims=6)


Data shape: (110073, 100)


In [5]:
print(train_loader.dataset.tensors[0][0][0])

tensor([0.0060, 0.0050, 0.0100, 0.0110, 0.0110, 0.0140, 0.0090, 0.0180, 0.0150,
        0.0150, 0.0190, 0.0170, 0.0160, 0.0140, 0.0190, 0.0230, 0.0180, 0.0210,
        0.0200, 0.0240, 0.0150, 0.0120, 0.0080, 0.0070, 0.0100, 0.0070, 0.0060,
        0.0080, 0.0080, 0.0090, 0.0130, 0.0080, 0.0130, 0.0160, 0.0180, 0.0160,
        0.0140, 0.0200, 0.0180, 0.0200, 0.0180, 0.0190, 0.0190, 0.0140, 0.0150,
        0.0160, 0.0160, 0.0150, 0.0200, 0.0190, 0.0160, 0.0150, 0.0170, 0.0180,
        0.0180, 0.0120, 0.0080, 0.0050, 0.0050, 0.0100, 0.0040, 0.0060, 0.0150,
        0.0220, 0.0230, 0.0260, 0.0220, 0.0260, 0.0250, 0.0220, 0.0210, 0.0170,
        0.0170, 0.0170, 0.0200, 0.0250, 0.0240, 0.0250, 0.0230, 0.0240, 0.0210,
        0.0180, 0.0130, 0.0100, 0.0000, 0.0030, 0.0010, 0.0000, 0.0050, 0.0070,
        0.0070, 0.0180, 0.0170, 0.0220, 0.0190, 0.0250, 0.0270, 0.0270, 0.0270,
        0.0230])


In [6]:
print(f"Length of train_loader: {len(train_loader.dataset)}")
print(f"Length of val_loader: {len(val_loader.dataset)}")

Length of train_loader: 99065
Length of val_loader: 11008


In [7]:
def hello(a,b,c):
    print(a,b,c)
    return a+b+c

abs = [1,2,3]
print(hello(*abs))

1 2 3
6


In [8]:
from torchsummary import summary
vae = vae.to('cuda')
print(vae)
summary(vae, (1, 100))

VAE(
  (encoder): Sequential(
    (0): Conv1d(1, 32, kernel_size=(3,), stride=(2,), padding=(31,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv1d(32, 64, kernel_size=(3,), stride=(2,), padding=(21,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): Conv1d(64, 128, kernel_size=(3,), stride=(2,), padding=(11,))
    (5): LeakyReLU(negative_slope=0.01)
    (6): Conv1d(128, 512, kernel_size=(4,), stride=(2,), padding=(5,))
    (7): LeakyReLU(negative_slope=0.01)
    (8): Flatten(start_dim=1, end_dim=-1)
    (9): Linear(in_features=12288, out_features=24, bias=True)
    (10): LeakyReLU(negative_slope=0.01)
  )
  (code_mean): Linear(in_features=24, out_features=6, bias=True)
  (code_std_dev): Linear(in_features=24, out_features=6, bias=True)
  (decoder): Sequential(
    (0): Linear(in_features=6, out_features=24, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=24, out_features=12288, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Unflatten(

## Train Model

In [9]:
vae.train_model(train_loader, n_epochs=10)  # Loss function might be wrong

Epoch 1/10, Loss: 342.9369812011719
Epoch 2/10, Loss: 342.9376220703125
Epoch 3/10, Loss: 342.9325256347656
Epoch 4/10, Loss: 342.9334716796875
Epoch 5/10, Loss: 342.9325866699219
Epoch 6/10, Loss: 342.9323425292969
Epoch 7/10, Loss: 342.9314270019531
Epoch 8/10, Loss: 342.9344482421875
Epoch 9/10, Loss: 342.9320068359375
Epoch 10/10, Loss: 342.929931640625


In [10]:
vae.save_model("Weights/vae.pth")

In [9]:
vae.load_model("Weights/vae.pth")

## Create LSTM data and model

In [15]:
encoded_training_data = vae.encode_data(train_loader)

In [16]:
print(encoded_training_data.shape)

(99040, 6)


In [17]:
# Create a DataLoader object
lstm_dataloader = DataLoaderGenerator(encoded_training_data, batch_size=32)
train_loader_lstm, val_loader_lstm = lstm_dataloader.generate()

In [18]:
train_loader_lstm.dataset.tensors[0].shape

torch.Size([89136, 1, 6])

In [22]:
from Utils.models import LSTM

lstm = LSTM()
lstm = lstm.to('cuda')
print(lstm)
summary(lstm, (1, 6))

TypeError: RNNBase.__init__() got an unexpected keyword argument 'return_sequences'

## Train LSTM

In [ ]:
lstm.train_model(train_loader_lstm, n_epochs=10)